<a href="https://colab.research.google.com/github/Smruthi3/END2/blob/main/Session7-Assignment/Assignment-Part2/Session7_Assignment_part2_quora_dup_ques_seq2seq_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

from torchtext.legacy import data

import spacy
import numpy as np

import random
import math
import time

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
#!pip install spacy --upgrade

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Preview : 
An important product principle for Quora is that there should be a single question page for each logically distinct question. As a simple example, the queries “What is the most populous state in the USA?” and “Which state in the United States has the most people?” should not exist separately on Quora because the intent behind both is identical. Having a canonical page for each logically distinct query makes knowledge-sharing more efficient in many ways: for example, knowledge seekers can access all the answers to a question in a single location, and writers can reach a larger readership than if that audience was divided amongst several pages.

The data released by quora conatins set of questions with alternative ways of asking the same questions along with tag which says wheather question is a duplicated question or not

The dataset is downloaded from http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv

Data set contains 6 columns out of which question1,question2 and is_duplicate fields are our interset. Only rows with is_duplicate =1 is considered




In [ ]:
import os 
import pandas as pd
DATA_DIR = 'drive/My Drive/END2/Session7-Assignment/Assignment-part2/'

In [ ]:
# import shutil
# shutil.unpack_archive(DATA_DIR+'Question_Answer_Dataset_v1.2.tar.gz', DATA_DIR+'QA_dataset')

In [ ]:
import glob

import pandas as pd

quora_duplicate_ques_dataset = pd.read_csv(DATA_DIR+"quora_duplicate_questions.tsv", sep='\t')

print(f' Total number of records or dimentions : {quora_duplicate_ques_dataset.shape}') 

 Total number of records or dimentions : (404290, 6)


In [ ]:
quora_duplicate_ques_dataset.head()


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
quora_duplicate_ques_dataset = quora_duplicate_ques_dataset[quora_duplicate_ques_dataset.is_duplicate==1]
quora_duplicate_ques_dataset = quora_duplicate_ques_dataset.reset_index(drop=True)

print(f'After selecting only duplicated questions, total number of records or dimentions : {quora_duplicate_ques_dataset.shape}') 

After selecting only duplicated questions, total number of records or dimentions : (149263, 6)


In [ ]:
# %%bash
# python -m spacy download en
# python -m spacy download de

In [ ]:
# spacy_de = spacy.load('de_core_news_sm')
# spacy_en = spacy.load('en_core_web_sm')

In [ ]:
# def tokenize_de(text):
#     """
#     Tokenizes German text from a string into a list of strings (tokens) and reverses it
#     """
#     return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

# def tokenize_en(text):
#     """
#     Tokenizes English text from a string into a list of strings (tokens)
#     """
#     return [tok.text for tok in spacy_en.tokenizer(text)]

### Defining Fields

In [ ]:
question = Field(tokenize = 'spacy', 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

dup_question = Field(tokenize = 'spacy', 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)


fields = [('questions', question),('duplicate_questions',dup_question)]

In [ ]:
quora_dup_ques_dataset_examples = [data.Example.fromlist([str(quora_duplicate_ques_dataset.question1[i]),str(quora_duplicate_ques_dataset.question2[i])],fields) for i in range(quora_duplicate_ques_dataset.shape[0])]

In [ ]:
quora_dup_ques_Dataset= data.Dataset(quora_dup_ques_dataset_examples, fields)

In [ ]:
(train,test) = quora_dup_ques_Dataset.split(split_ratio=[0.70, 0.30], random_state=random.seed(SEED))

In [ ]:
print(f"Toatal examples: {len(quora_dup_ques_dataset_examples)}")
print(f"Number of training examples: {len(train.examples)}")
print(f"Number of testing examples: {len(test.examples)}")

Toatal examples: 149263
Number of training examples: 104484
Number of testing examples: 44779


In [ ]:
print(vars(train.examples[1]))

{'questions': ['what', 'are', 'the', 'basic', 'types', 'of', 'satellites', '?'], 'duplicate_questions': ['what', 'are', 'different', 'types', 'of', 'satellites', '?', 'what', 'is', 'the', 'most', 'advanced', 'type', '?']}


### Building Vocabulary

In [ ]:
question.build_vocab(train, min_freq = 2)
dup_question.build_vocab(train, min_freq = 2)

In [ ]:
print(f"Unique tokens in question vocabulary: {len(question.vocab)}")
print(f"Unique tokens in answer vocabulary: {len(dup_question.vocab)}")

Unique tokens in question vocabulary: 14522
Unique tokens in answer vocabulary: 14469


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE = 128

train_iterator, test_iterator = data.BucketIterator.splits(
    (train, test), 
    batch_size = BATCH_SIZE, 
    sort_key= lambda x: len(x.questions),
    sort_within_batch=True,
    device = device)

### Defining The model arrchitecture

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [ ]:
INPUT_DIM = len(question.vocab)
OUTPUT_DIM = len(dup_question.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(14522, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(14469, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=14469, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 22,200,709 trainable parameters


# Model Training and Evaluation

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = dup_question.vocab.stoi[dup_question.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

### Training loop

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        ques = batch.questions
        ans = batch.duplicate_questions
        
        optimizer.zero_grad()
        
        output = model(ques, ans)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        ans = ans[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, ans)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

### Evaluation loop

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            ques = batch.questions
            ans = batch.duplicate_questions

            output = model(ques, ans, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            ans = ans[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, ans)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### Model training and Evaluation

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_test_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    test_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save(model.state_dict(), DATA_DIR+'Quora-duplicate-ques-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {test_loss:.3f} |  Val. PPL: {math.exp(test_loss):7.3f}')

Epoch: 01 | Time: 4m 7s
	Train Loss: 3.818 | Train PPL:  45.503
	 Val. Loss: 4.126 |  Val. PPL:  61.936
Epoch: 02 | Time: 4m 9s
	Train Loss: 3.302 | Train PPL:  27.154
	 Val. Loss: 3.814 |  Val. PPL:  45.337
Epoch: 03 | Time: 4m 10s
	Train Loss: 2.975 | Train PPL:  19.586
	 Val. Loss: 3.736 |  Val. PPL:  41.910
Epoch: 04 | Time: 4m 9s
	Train Loss: 2.737 | Train PPL:  15.442
	 Val. Loss: 3.658 |  Val. PPL:  38.779
Epoch: 05 | Time: 4m 9s
	Train Loss: 2.551 | Train PPL:  12.821
	 Val. Loss: 3.601 |  Val. PPL:  36.633
Epoch: 06 | Time: 4m 10s
	Train Loss: 2.401 | Train PPL:  11.039
	 Val. Loss: 3.616 |  Val. PPL:  37.203
Epoch: 07 | Time: 4m 9s
	Train Loss: 2.306 | Train PPL:  10.033
	 Val. Loss: 3.576 |  Val. PPL:  35.732
Epoch: 08 | Time: 4m 9s
	Train Loss: 2.201 | Train PPL:   9.033
	 Val. Loss: 3.562 |  Val. PPL:  35.245
Epoch: 09 | Time: 4m 8s
	Train Loss: 2.120 | Train PPL:   8.328
	 Val. Loss: 3.597 |  Val. PPL:  36.473
Epoch: 10 | Time: 4m 9s
	Train Loss: 2.056 | Train PPL:   7.81